In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.ie.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from apscheduler.schedulers.blocking import BlockingScheduler

import pymysql
from datetime import datetime, timedelta
from google_trans_new import google_translator
import time
import random

host = '10.55.23.101'
port = 33060
user = 'root'
passwd = "1234"
db = 'idap'

showPrintMSG = 1 # 0:不呈現，1:呈現 debug mode
archiveDate = 10
web = 'chosun'
language = 'ko'
sleep_sec = 20
lang_src=language
lang_tgt='zh-tw'


In [101]:
# step 1. 取得4大類新聞url, 20210215
# politics, national, industry, international
def chosun_1(archiveDate = 10) :
    if showPrintMSG :
        print('start chosun_1 ',datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
    cur = conn.cursor()    

    tagList = {'politics':'politics', 'national':'national', 'industry':'economy', 'international':'international'}
    baseUrl = 'https://www.chosun.com/'
    deleteFromDate = datetime.strftime(datetime.now() - timedelta(archiveDate), '%Y%m%d')

    option = webdriver.ChromeOptions()
    #option.add_argument("--test-type");
    option.add_argument("--disable-gpu");
    #option.add_argument("--no-first-run");
    #option.add_argument("--no-default-browser-check");
    #option.add_argument("--ignore-certificate-errors");
    option.add_argument("--start-maximized");  
    
    #if not showPrintMSG :
    #    option.add_argument('--headless') 
    driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
     
     
    for tag in tagList :
        if showPrintMSG :
            print(tag)
        driver.get(baseUrl+tagList[tag])    
        #driver.implicitly_wait(30)        
        #WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.ID, "fusion-template-script")))
        
        for i in range(1,16) :
            creationdate = datetime.now()
            try :
                j = '1'
                url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
                title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
            except Exception as e:        
                j = '2'
                url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
                title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
            urlTmp = url.split('/')
            publishdate = urlTmp[len(urlTmp)-5]+urlTmp[len(urlTmp)-4]+urlTmp[len(urlTmp)-3]
            if showPrintMSG :
                print(i,title,',',publishdate,',',url)
                
            if int(publishdate) > int(deleteFromDate) :
                cur.execute('select count(1) from news_daily_source where url=%s',(url))
                if cur.fetchone()[0] == 0 :
                    title_tw = gl_translator(title, lang_src, lang_tgt)
                    if showPrintMSG :
                        print(i,title_tw)
                    cur.execute('insert into news_daily_source(web, title, title_tw, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, title_tw, publishdate, url, creationdate,language,tag, url))
                    cur.execute('commit')
                    time.sleep(sleep_sec)    
    cur.close()
    conn.close()    
    driver.close()
    
# step 2. 取得全文並翻譯, 20210217
def chosun_2() :
    
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
    cur = conn.cursor()
    cur1 = conn.cursor()

    option = webdriver.ChromeOptions()
    option.add_argument("--disable-gpu");
    option.add_argument("--start-maximized");    
    driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
    cur.execute('select url from news_daily_source where web="chosun" and status=0 and content is null')
    for r in cur :
        url = r[0]   
        if showPrintMSG :
            print(url)
        driver.get(url)
        try :
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "article-body")))
            content = driver.find_element_by_class_name('article-body').text
            content_tw =  gl_translator(content, lang_src, lang_tgt)
        except Exception as e:
            continue
            
        cur1.execute('update news_daily_source set content=%s,content_tw=%s where url=%s',(content,content_tw,url))
        cur1.execute('commit')
        time.sleep(sleep_sec)
    cur.close()
    cur1.close()
    conn.close()    
    driver.close()    

def delete_old_data(archiveDate = 10) :
    conn = pymysql . connect ( host = host , port = port , user = user , passwd = passwd , db = db ) 
    cur = conn . cursor ()    
    deleteFromDate = datetime.strftime(datetime.now() - timedelta(archiveDate), '%Y%m%d')
    cur.execute("delete from news_daily_source WHERE publishdate<=%s",(deleteFromDate))
    cur.execute("commit")

def gl_translator(text, lang_src, lang_tgt) :
    url_suffix_list = ['ac','ad','ae','al','am','as','at','az','ba','be','bf','bg','bi','bj','bs','bt','by','ca','cat','cc','cd','cf','cg','ch','ci','cl','cm','cn','co.ao','co.bw','co.ck','co.cr','co.id','co.il','co.in','co.jp','co.ke','co.kr','co.ls','co.ma','co.mz','co.nz','co.th','co.tz','co.ug','co.uk','co.uz','co.ve','co.vi','co.za','co.zm','co.zw','co','com.af','com.ag','com.ai','com.ar','com.au','com.bd','com.bh','com.bn','com.bo','com.br','com.bz','com.co','com.cu','com.cy','com.do','com.ec','com.eg','com.et','com.fj','com.gh','com.gi','com.gt','com.hk','com.jm','com.kh','com.kw','com.lb','com.lc','com.ly','com.mm','com.mt','com.mx','com.my','com.na','com.ng','com.ni','com.np','com.om','com.pa','com.pe','com.pg','com.ph','com.pk','com.pr','com.py','com.qa','com.sa','com.sb','com.sg','com.sl','com.sv','com.tj','com.tr','com.tw','com.ua','com.uy','com.vc','com.vn','com','cv','cx','cz','de','dj','dk','dm','dz','ee','es','eu','fi','fm','fr','ga','ge','gf','gg','gl','gm','gp','gr','gy','hn','hr','ht','hu','ie','im','io','iq','is','it','je','jo','kg','ki','kz','la','li','lk','lt','lu','lv','md','me','mg','mk','ml','mn','ms','mu','mv','mw','ne','nf','nl','no','nr','nu','pl','pn','ps','pt','ro','rs','ru','rw','sc','se','sh','si','sk','sm','sn','so','sr','st','td','tg','tk','tl','tm','tn','to','tt','us','vg','vu','ws']
    isSuccess = False
    url_suffix_inx = random.randint(0, len(url_suffix_list)-1)
    content_tw = ''
    while isSuccess == False :
        try :
            translator = google_translator(url_suffix=url_suffix_list[url_suffix_inx])        
            content_tw = translator.translate(text, lang_src=lang_src, lang_tgt=lang_tgt)
            isSuccess = True
        except Exception as e:
            url_suffix_inx = random.randint(0, len(url_suffix_list)-1)
    return content_tw

def chosun_etl():
    delete_old_data()
    chosun_1()
    chosun_2()   

In [28]:
scheduler = BlockingScheduler() 
#scheduler.add_job(news_ner, 'interval', seconds=10800) #3 hours 
#scheduler.add_job(news_etl, 'cron', hour='*/1', minute=20)
scheduler.add_job(chosun_etl, 'cron', hour='7-20,23,3,6', minute=40)
scheduler.start()

C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='main']/div[4]/section/div/div/div[2]/div[1]/div/div/div/div[1]/div[2]/div[1]/a"}
  (Session info: headless chrome=85.0.4183.83)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.17134 x86_64)


In [37]:
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()
cur1 = conn.cursor()
translator = google_translator() 
cur.execute('select url,title from news_daily_source where web="chosun" and status=0 and content is not null')
for r in cur :
    url = r[0]
    title = r[1]
    title_tw = translator.translate(title,lang_src='ko', lang_tgt='zh-tw')
    cur1.execute('update news_daily_source set title_tw=%s where url=%s',(title_tw,url))
    cur1.execute('commit')
    

In [46]:
showPrintMSG = True
not showPrintMSG 

False

In [11]:
# Economy, 20210215
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()
web = 'chosun'
language = 'ko'
tag = 'industry'
targetUrl = 'https://www.chosun.com/economy/'

option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
#option.add_argument('headless') 
driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
driver.get(targetUrl) 
creationdate = datetime.now()

for i in range(1,16) :
    try :
        j = '1'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    except Exception as e:        
        j = '2'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    urlTmp = url.split('/')
    if int(urlTmp[5]) > 2020 :
        publishdate = urlTmp[5]+urlTmp[6]+urlTmp[7]
    else :
        publishdate = urlTmp[4]+urlTmp[5]+urlTmp[6]
    
    print(i,title,',',publishdate,',',url)
    cur.execute('insert into news_daily_source(web, title, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, publishdate, url, creationdate,language,tag, url))
    cur.execute('commit')
    #res = requests.get(url)
    #res.encoding = 'utf-8'
    #print(res.status_code)
    #soup = BeautifulSoup(res.text, 'html.parser')
    #soup
cur.close()
conn.close()    
driver.close()

C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


1 그게 왜 커피 머신에서 나와 , 이태리 사람들 입이 떡 벌어졌다 , 20210217 , https://www.chosun.com/economy/startup_story/2021/02/17/K7PDEREG5ZBOPI7LQHJGBED54Q/
2 ‘변이'와 ‘변이'가 만났다, 독성 강한 수퍼 코로나 나오나 , 20210217 , https://www.chosun.com/economy/science/2021/02/17/4UAJQQK2HRG4ROUCKNPXLQUZA4/
3 대출이 월세보다 싸도 집 안산다...일본만 집값 잠잠한 이유 , 20210216 , https://www.chosun.com/economy/real_estate/2021/02/16/WJUOBS2A7VEPDOG7UBPDFAA7DQ/
4 현대車, 잇단 전기차 화재… 코나 배터리 전량 교체로 가닥 , 20210217 , https://www.chosun.com/economy/auto/2021/02/17/W7VX5GPB4BADLIXEUGMYUM73SA/
5 금보다 비트코인... 1년새 850% 급등, 5만달러도 뚫었다 , 20210216 , https://www.chosun.com/economy/2021/02/16/DU7BMJVJB5DHHA7FXIPNJ2L7JQ/
6 홍남기 “서울 아파트 큰폭 조정 있었다, 냉철하게 시장 봐야” , 20210217 , https://www.chosun.com/economy/2021/02/17/KIG4VIYFFNGD5AVUNYN4JP63SY/
7 포스코가 만든 수소로 현대차 달린다 , 20210217 , https://www.chosun.com/economy/economy_general/2021/02/17/7PVJBXV6HFE7HPS4BIUZFDCBRA/
8 “물량 좀 풀어줘!” 머스크가 콕 찍었다, ‘도지코인'이 뭐길래 , 20210216 , https://www.chosun.com/economy/int_economy/2021/02/16/O2WZ4ZM34ZAGDHGSBV

In [13]:
# Politics, 20210215
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()
web = 'chosun'
language = 'ko'
tag = 'politics'
targetUrl = 'https://www.chosun.com/politics/'

option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
#option.add_argument('headless') 
driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
driver.get(targetUrl) 
creationdate = datetime.now()

for i in range(1,16) :
    try :
        j = '1'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
                                           #//*[@id="main"]/div[4]/section/div/div/div[2]/div[1]         /div/div/div/div[1]/div[1]    /div[1]/a
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    except Exception as e:        
        j = '2'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    urlTmp = url.split('/')
    if int(urlTmp[5]) > 2020 :
        publishdate = urlTmp[5]+urlTmp[6]+urlTmp[7]
    else :
        publishdate = urlTmp[4]+urlTmp[5]+urlTmp[6]
    
    print(i,title,',',publishdate,',',url)
    cur.execute('insert into news_daily_source(web, title, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, publishdate, url, creationdate,language,tag, url))
    cur.execute('commit')
cur.close()
conn.close()    
driver.close()

C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


1 신현수, 文대통령 반려에도 사표 또 냈다 , 20210217 , https://www.chosun.com/politics/2021/02/17/EYKVD2LDGNDNHIKIRQQUXT7MSY/
2 군 “민통선서 붙잡은 1명, 북한 남성 추정” ‘노크귀순’ 동해 또 뚫려 , 20210216 , https://www.chosun.com/politics/2021/02/16/2HDXNQ4TTVCJJPG5USBGISYFIA/
3 리설주 1년여 만에 공개석상…김정은과 기념공연 관람 , 20210217 , https://www.chosun.com/politics/north_korea/2021/02/17/MTTRJ7OG7RB7HFBLN6NOAYOUKM/
4 文, 신현수 수석 빼놓고 이광철 비서관과 검찰 인사 , 20210217 , https://www.chosun.com/politics/2021/02/17/SUIFJ6ID7FFVVLGYIPWARUSFYU/
5 이번엔 ‘산책 귀순’...22사단 석달 만에 또 뚫려 , 20210217 , https://www.chosun.com/politics/diplomacy-defense/2021/02/17/52Q5DJ46QJG4PCPKKHJF6SR46U/
6 이광철, 조국 보좌한 민변 출신… 선거개입 관련 검찰조사 받아 , 20210217 , https://www.chosun.com/politics/blue_house/2021/02/17/AUHCQHX3ARBMXNJ7CQT6XVD3NE/
7 조은산, 교황 소환한 이재명·임종석 논쟁에 “한국 일이나 살펴라” [전문] , 20210216 , https://www.chosun.com/politics/politics_general/2021/02/16/ZY6ISRAB5BA3ZMZY7ZEVAO2UII/
8 文대통령 딸 다혜씨 “대통령 외손자 방역지침 위반 일절 없다” , 20210216 , https://www.chosun.com/politics/2021/02/16/NU7G5QGXUNEEBLW3NM

In [21]:
# National, 20210215
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
cur = conn.cursor()
web = 'chosun'
language = 'ko'
tag = 'national'
targetUrl = 'https://www.chosun.com/national/'

option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
#option.add_argument('headless') 
driver = webdriver.Chrome(r"D:\chromedriver.exe",chrome_options=option)
driver.get(targetUrl) 
creationdate = datetime.now()

for i in range(1,16) :
    try :
        j = '1'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
                                           #//*[@id="main"]/div[4]/section/div/div/div[2]/div[1]         /div/div/div/div[1]/div[1]    /div[1]/a
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    except Exception as e:        
        j = '2'
        url = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").get_attribute('href')
        title = driver.find_element_by_xpath("//*[@id='main']/div[4]/section/div/div/div[2]/div["+str(i)+"]/div/div/div/div[1]/div["+j+"]/div[1]/a").text
    urlTmp = url.split('/')    
    publishdate = urlTmp[len(urlTmp)-5]+urlTmp[len(urlTmp)-4]+urlTmp[len(urlTmp)-3]    
    print(i,title,',',publishdate,',',url)
    cur.execute('insert into news_daily_source(web, title, publishdate, url, creationdate,language,tag) SELECT * FROM (SELECT %s, %s, %s, %s, %s, %s, %s) AS tmp WHERE NOT EXISTS (SELECT url FROM news_daily_source WHERE url = %s) LIMIT 1',(web, title, publishdate, url, creationdate,language,tag, url))
    cur.execute('commit')
cur.close()
conn.close()    
driver.close()

C:\Users\jacob.liang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


1 [단독] 이성윤 중앙지검장 전격 소환 통보... 조사 불응 , 20210217 , https://www.chosun.com/national/court_law/2021/02/17/ADQ27PVWYNGFXJ2FPLF7R37Y34/
2 전세계 확진 70% 급감, 희망이 보인다 , 20210217 , https://www.chosun.com/national/welfare-medical/2021/02/17/F6RYOTLZPVEVNPKUPAAAT6LZJM/
3 법무부, 이재용에 취업제한 통보...옥중 경영 못할 수도 , 20210217 , https://www.chosun.com/national/court_law/2021/02/17/Q4UUUHBQ65EXPLOWSMUYXH3SRQ/
4 60대 얼굴에 ‘4연발 니킥'... 아들뻘 그놈, 모텔에 숨어있었다 , 20210216 , https://www.chosun.com/national/incident/2021/02/16/XAYGQQU22VGIRO6ESDQWWCJ2HQ/
5 법원 “대법원 앞 김명수 규탄집회 허용”... 경찰 금지 결정 뒤집었다 , 20210216 , https://www.chosun.com/national/court_law/2021/02/16/GTUK2DL7MBFLXHJ4DS7OMZKMPY/
6 “샤워하는 알몸이 훤히...” 그랜드 조선 제주 사우나 논란 , 20210216 , https://www.chosun.com/national/2021/02/16/6PBMK4GXFJFFFLVXODNNFMAROM/
7 노래방 꽉 차고 포차 앞 대기줄... 지방 ‘2차’가 살아났다 , 20210216 , https://www.chosun.com/national/national_general/2021/02/16/6CAIZICIVBDITIGDXFFCBLE5LI/
8 여교사, 학폭 피해 위로한다며 중학생 성폭행... 징역 3년 선고 , 20210216 , https://www.chosun.com/national/regiona